In [2]:
from ultralytics import YOLO
import torch, torchvision
import cv2 as cv
import numpy as np

In [3]:

# Load the weights directly
model = YOLO('./train/weights/best.pt')

results = model('./gateRender1_903.jpg',device='mps')



image 1/1 /Users/michalkozicki/yolo-trained/gateRender1_903.jpg: 640x640 1 gate, 19.2ms
Speed: 8.7ms preprocess, 19.2ms inference, 98.2ms postprocess per image at shape (1, 3, 640, 640)


In [4]:
import pathlib
import time
val_src = pathlib.Path('/Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val')
times = []
results = model(val_src)




image 1/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_1.jpg: 640x640 1 gate, 19.5ms
image 2/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_101.jpg: 640x640 1 gate, 11.5ms
image 3/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_115.jpg: 640x640 1 gate, 12.5ms
image 4/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_126.jpg: 640x640 1 gate, 13.0ms
image 5/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_132.jpg: 640x640 1 gate, 15.0ms
image 6/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_135.jpg: 640x640 1 gate, 18.6ms
image 7/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_137.jpg: 640x640 1 gate, 11.6ms
image 8/186 /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_14.jpg: 640x640 1 gate, 12.8ms
image 9/186 /Users

In [5]:
reference_path = "./res/gate_ref.jpg"
ref_gray = cv.cvtColor(cv.imread(reference_path), cv.COLOR_BGR2GRAY)

In [6]:
import math
def rotationMatrixToEulerAngles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])



In [7]:

cv.imwrite('./res/gate_Front.jpg', cv.imread('./res/Gate teksture_A2RL X DCL-2.png'), [int(cv.IMWRITE_JPEG_QUALITY), 95]) 
cv.imwrite('./res/gate_Back.jpg', cv.imread('./res/Gate teksture_A2RL X DCL-2_mirror.png'), [int(cv.IMWRITE_JPEG_QUALITY), 95]) 

libpng warning: iCCP: known incorrect sRGB profile


True

In [8]:
# Load the texture images /Users/michalkozicki/yolo-trained/res/Gate teksture_A2RL X DCL-2.png

texture_front = cv.imread('./res/gate_Front.jpg', cv.IMREAD_GRAYSCALE)

# Initialize a feature detector (e.g., SIFT)
sift = cv.SIFT_create()

# Detect keypoints and compute descriptors for the front texture
keypoints_ref_texture, descriptors_ref_texture = sift.detectAndCompute(texture_front, None)


In [10]:
#LOADING REFERENCE FROM 3D model
key_points_3d = []
with open('./res/3d_points.txt', 'r') as f:
    for line in f:
        x, y, z = map(float, line.strip().split())
        key_points_3d.append([x, y, z])

key_points_3d = np.array(key_points_3d, dtype=np.float32)

In [18]:
#program runs for all results
angles=[]
for result in results:
    path = result.path
    if result.boxes.xyxy.shape[0] == 0:
        print(f"No bounding box found for image {path}")
        continue
    #IMAGE PREPROCESSING
    x_1, y_1, x_2, y_2 = map(int, result.boxes.xyxy[:,:4][0]) #getting xyxy values for bounding box
    image_path=path
    gray_image = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2GRAY) #converting to grayscale
    if gray_image is None:
        print(f"Error: Could not read image at {image_path}")
        continue
    
    cropped_image = gray_image[y_1:y_2, x_1:x_2] #cropping to bounding box
    
    
    #ANALYSIS OF IMAGE
    sift = cv.SIFT_create() #extracting keypoints from the image being processed
    keypoints_photo, descriptors_photo = sift.detectAndCompute(cropped_image, None)
    #assuming you have texture
    bf = cv.BFMatcher()
    
    if descriptors_photo is None:
        print("No descriptors found in the reference texture.")
        continue
    if descriptors_ref_texture.dtype != np.float32:
        descriptors_ref_texture = descriptors_ref_texture.astype(np.float32)    
    matches = bf.knnMatch(descriptors_ref_texture, descriptors_photo, k=2) #using the already determined keypoints and descriptors for reference photo
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good_matches.append(m)
    object_points = np.float32([key_points_3d[m.queryIdx] for m in good_matches if m.queryIdx < len(key_points_3d)])
    image_points = np.float32([keypoints_photo[m.trainIdx].pt for m in good_matches if m.queryIdx < len(key_points_3d)])
    #TBD the camera config
    fx = fy =550
    cx = 1640
    cy = 1232
    camera_matrix = np.array([[fx, 0, cx],
                              [0, fy, cy],
                              [0,  0,  1]], dtype=np.float64)

    dist_coeffs = np.zeros((4,1))  #distortion coefficients for camera, rn unknown
    
    #CALCULATING ROTATION
    success, rotation_vector, translation_vector = cv.solvePnP(
    object_points,
    image_points,
    camera_matrix,
    dist_coeffs,
    flags=cv.SOLVEPNP_ITERATIVE
    )
    rotation_matrix, _ = cv.Rodrigues(rotation_vector)
    #TRANSLATING TO RPY
    euler_angles = rotationMatrixToEulerAngles(rotation_matrix)
    pitch, yaw, roll = np.degrees(euler_angles)
    angles.append((result, (pitch, yaw, roll)))

No descriptors found in the reference texture.
No bounding box found for image /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_23.jpg
No bounding box found for image /Users/michalkozicki/yolo-trained/drone-vision/datasets/images/val/gateRender1_444.jpg


In [19]:
angles[10][0].show()
print(angles[10][1])

(139.89744347167058, -40.054426466521576, -174.6469854129691)
